In [1]:
# img = [].
# for file in img_files:
#     image = Image.open(file)
#     img.append(image)
# # images = [Image.open(file) for file in img_files]

# columns = 5
# rows = len(img) // columns + int(len(img) % columns !=0)

# fig, axes = plt.subplots(rows, columns, figsize=(15,15))
# axes = axes.flatten()

# for idx, image in enumerate(img):
#     axes[idx].imshow(image, cmap='gray')
#     axes[idx].axis('off')
#     axes[idx].set_title(f"Image {idx+1}")

# for ax in axes[len(img):]:
#     ax.axis('off')



# print(f"found {len(img_files)} .pgm images")
# plt.tight_layout()
# plt.show()
#####################################################################
# cols = 5
# rows = len(images) // cols + int(len(images) % cols != 0)

# # Create subplots
# # fig, axes = plt.subplots(rows, cols, figsize=(15, 15))
# fig, axes = plt.subplots(rows, cols, figsize=(15, 15))

# axes = axes.flatten()  # Flatten in case of 2D array of axes

# # Loop through images and plot each one
# for idx, img in enumerate(images):
#     axes[idx].imshow(img, cmap='gray')
#     axes[idx].axis('off')
#     axes[idx].set_title(f"Image {idx+1}")

# # Turn off any unused subplots
# for ax in axes[len(images):]:
#     ax.axis('off')

# plt.tight_layout()
# plt.show()

In [4]:
# reading the Image file 
img_files = glob.glob("all-mias")
# img_files = "all-mias"
print(f"found {len(img_files)} .pmg images")

# reading the txt file 
txt_path = "info.txt"
with open(txt_path, "r") as file:
    txt_file = file.readlines()

first_img = Image.open(img_files[11])
plt.imshow(first_img, cmap='gray')
plt.title("Example .pgm Image")
plt.axis('off')  # Hide axis ticks
plt.show()